In [ ]:
"""Example for plotting gradient data"""
import os.path as op
from glob import glob
import itertools

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt

from utils import plot_gradient, plot_subcortical_gradient

In [ ]:
import numpy as np
import nibabel as nib
import pandas as pd
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np

class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

# Visualize Gradients

## Margulies 2016 (HCP S900)

In [ ]:

from neuromaps.datasets import fetch_annotation

data_dir = "../data"
neuromaps_dir = op.join(data_dir, "neuromaps-data")

marg_grad_out_path = "../figures/Fig/gradient"
grad_marg_lh, grad_marg_rh = fetch_annotation(
    source="margulies2016",
    desc="fcgradient01",
    space="fsLR",
    den="32k",
    data_dir=neuromaps_dir,
)

plot_gradient(data_dir, [(grad_marg_lh, grad_marg_rh)], cbar=True, out_dir=marg_grad_out_path)

## Peraza 2022 (HCP S1200)

In [ ]:
jperaza_grad_path = op.abspath("../results/gradient/GIFTI-NIfTI_files")
jperaza_grad_out_path = "../figures/Fig/gradient"
jperaza_grad_lh_fnames = sorted(glob(op.join(jperaza_grad_path, "*hemi-L_feature.func.gii")))
jperaza_grad_rh_fnames = sorted(glob(op.join(jperaza_grad_path, "*hemi-R_feature.func.gii")))
jperaza_subcort_grad_fnames = sorted(glob(op.join(jperaza_grad_path, "*_feature.nii.gz")))
jperaza_grad_fnames = zip(jperaza_grad_lh_fnames, jperaza_grad_rh_fnames)

plot_gradient("../data", jperaza_grad_fnames, cbar=False, layout='column', out_dir=jperaza_grad_out_path)
# plot_subcortical_gradient(jperaza_subcort_grad_fnames, threshold_=0.01)

In [ ]:
lambdas = np.load("../results/gradient/lambdas.npy")
lambdas = lambdas[:9]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("ticks",{'axes.grid' : True})
# sns.set(style="white")

_, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(3.5, 6))
#ax1.set_xlabel("Component Nb")


ax1.set_ylabel("Explained Variance Ratio")
ax2.set_ylabel("Difference in Explained\nVariance Ratio")
x1 = range(1, lambdas.size + 1, 1)
x2 = range(1, lambdas.size, 1)
y1 = (100 * lambdas) / lambdas.sum()
y2 = 100 * (lambdas[:-1] - lambdas[1:]) / lambdas.sum()

sns.lineplot(
    x=x1,
    y=y1,
    marker="o",
    color="#1D5B79",
    ax=ax1,
)
sns.lineplot(
    x=x2,
    y=y2,
    marker="o",
    color="#1D5B79",
    ax=ax2,
)
ax1.set_xticks(x1)
ax1.set_xticklabels(x1)
ax1.set_xlabel("Component")

x2_lbs = [f"{x}-{x+1}" for x in x2]
ax2.set_xticks(x2)
ax2.set_xticklabels(x2_lbs) 
ax2.set_xlabel("Component Difference ($C_{i} - C_{i+1}$)")

plt.tight_layout()
plt.savefig(op.join("./", "Fig", "Fig-S3.png"), bbox_inches="tight", dpi=1000)
plt.show()


In [ ]:
sum(y1[:4])

In [ ]:
gradients = np.load("../results/gradient/gradients.npy")

In [ ]:
template_dir = "../data/templates"
subcortical_fn = op.join(template_dir, "rois-subcortical_mni152_mask.nii.gz")
subcort_img = nib.load(subcortical_fn)

full_vertices = 64984
hemi_vertices = full_vertices // 2

subcort_dat = subcort_img.get_fdata()
subcort_mask = subcort_dat != 0
n_subcort_vox = np.where(subcort_mask)[0].shape[0]

n_gradients = gradients.shape[1]
grad_lst = []
for i in range(n_gradients):
    cort_grads = gradients[: gradients.shape[0] - n_subcort_vox, i]
    grad_lst.append(cort_grads)

In [ ]:
labels_00 = list(np.load("./temp_segmentation/KMeans_00-04_labels.npy"))
labels_01 = list(np.load("./temp_segmentation/KMeans_01-04_labels.npy"))
labels_02 = list(np.load("./temp_segmentation/KMeans_02-04_labels.npy"))
labels_03 = list(np.load("./temp_segmentation/KMeans_03-04_labels.npy"))
labels_04 = list(np.load("./temp_segmentation/KMeans_04-04_labels.npy"))
labels_05 = list(np.load("./temp_segmentation/KMeans_05-04_labels.npy"))
labels_06 = list(np.load("./temp_segmentation/KMeans_06-04_labels.npy"))
labels_07 = list(np.load("./temp_segmentation/KMeans_07-04_labels.npy"))

In [ ]:
grad_df = pd.DataFrame(np.array(grad_lst).T, columns=[f"Gradient {i+1}" for i in range(n_gradients)])
grad_df["label_00"] = labels_00
grad_df["label_01"] = labels_01
grad_df["label_02"] = labels_02
grad_df["label_03"] = labels_03
grad_df["label_04"] = labels_04
grad_df["label_05"] = labels_05
grad_df["label_06"] = labels_06
grad_df["label_07"] = labels_07
grad_df

In [ ]:
sns.set(style="whitegrid", font_scale=2)
pal = ["#0174BE", "#008170", "#D0D4CA", "#B31312"] #  "#ED7D31"
n_rows, n_columns = 4, 4
vars = [f"Gradient {i+1}" for i in range(9)]

for row, column in itertools.product(range(n_rows), range(n_columns)):
    if row < column:
        x, y = vars[column], vars[row]
        figure = plt.figure(figsize=(2, 2))
        # figure, ax = plt.subplots(figsize=(3, 3))

        g = sns.jointplot(
            data=grad_df, 
            x=x, 
            y=y, 
            hue="label", 
            palette=pal, 
            alpha=1, 
            s=3,
            legend=False,
        )
        g.ax_joint.collections[0].set_edgecolor('none')
        g.ax_joint.collections[0].set_linewidth(0)
   
        if row == 0:
            g.ax_marg_y.set_ylim(-5, 7)
        if row == 1:
            g.ax_marg_y.set_ylim(-3, 3)
        if row == 2:
            g.ax_marg_y.set_ylim(-2, 4)
        if row == 3:
            g.ax_marg_y.set_ylim(-2, 2)
        if column == 1:
            g.ax_marg_x.set_xlim(-3, 3)
        if column == 2:
            g.ax_marg_x.set_xlim(-2, 4)
        if column == 3:
            g.ax_marg_x.set_xlim(-2, 2)


        plt.savefig(f"./temp_fig/{row}-{column}_mutidimensional.png", dpi=100)
        plt.show()

In [ ]:
pal = ["#0174BE", "#008170", "#D0D4CA", "#B31312"]
for comp in [0, 1, 2, 3, 4, 5, 6, 7]:
    figure = plt.figure(figsize=(2, 2))
    g = sns.scatterplot(
        data=grad_df, 
        x="Gradient 3", 
        y="Gradient 1", 
        hue=f"label_{comp:02d}", 
        palette=pal, 
        alpha=1, 
        s=3,
        edgecolors=None,
        linewidth=0,
        legend=False,
    )
    g.set_axis_off()
    plt.savefig(f"./temp_fig/2D-gradient_G1:G{comp+1}.png", bbox_inches="tight", dpi=1000)

In [ ]:
from sklearn import metrics
# comps = [0, 1, 2, 3, 4, 5, 6, 7]
rows = []
for i in range(8):
    columns = []
    for j in range(8):
        labels_pred = grad_df[f"label_{i:02d}"].to_list()
        labels_true = grad_df[f"label_{j:02d}"].to_list()
        nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
        columns.append(nmi)
    rows.append(columns)
corr = np.array(rows)

In [ ]:
label_list = ["G1", "G1:G2", "G1:G3", "G1:G4", "G1:G5", "G1:G6", "G1:G7", "G1:G8"]

In [ ]:
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(7, 5))
    ax = sns.heatmap(corr, mask=mask, vmin=0, vmax=1, annot=True, square=True)

    ax.set_xticklabels(label_list)
    ax.set_yticklabels(label_list)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=14)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=14)
plt.savefig(f"./temp_fig/2D-gradient_NMI.png", bbox_inches="tight", dpi=1000)

In [ ]:
import numpy as np

from surfplot.utils import add_fslr_medial_wall
from matplotlib.colors import ListedColormap
from neuromaps.datasets import fetch_fslr
from surfplot import Plot
from brainspace.datasets import load_parcellation

surfaces = fetch_fslr()
lh, rh = surfaces['inflated']

full_vertices = 64984
hemi_vertices = full_vertices // 2
prin_grad = add_fslr_medial_wall(np.array(labels))  # Add medial wall for plotting
labels_lh, labels_rh = prin_grad[:hemi_vertices], prin_grad[hemi_vertices:full_vertices]

pal = ["#0174BE", "#008170", "#D0D4CA", "#B31312"]
for region in range(4):
    # zero-out all regions except 71 and 72
    map_lh = np.zeros_like(labels_lh)
    map_lh[labels_lh==region] = 1
    map_rh = np.zeros_like(labels_rh)
    map_rh[labels_rh==region] = 1

    cmap = ListedColormap(pal[region], 'regions', N=1)
    p = Plot(rh, views='medial')
    p.add_layer(map_rh, cmap=cmap, cbar=False)
    p.add_layer(map_rh, cmap='Greys', as_outline=True, cbar=False)

    fig = p.build()
    plt.savefig(f"./temp_fig/{region}-gradient.tiff", bbox_inches="tight", dpi=500)


In [ ]:

n_rows, n_columns = 4, 4
figure1 = plt.figure(figsize=(12, 12))
gs = GridSpec(
    3, 
    3, 
    figure=figure1, 
)

for row in range(n_rows):
    for column in reversed(range(n_columns)):
        if row < column:
            print(row, column, row*(n_columns-1) + column - 1)
            # fig = plt.figure(figsize=(4, 4))
            img1 = mpimg.imread(f"./temp_fig/{row}-{column}_mutidimensional.png")

            ax = figure1.add_subplot(gs[row*(n_columns-1) + column - 1], aspect="equal")
            # gs.update(left=0.55, right=0.98, hspace=0.05)
            ax.imshow(img1)
            # if gradient_row == 3:
            #    ax1.set_title('Gradient {}'.format(i))
            ax.set_axis_off()

plt.subplots_adjust(wspace=-0.25, hspace=-0.18)
plt.savefig(f"./temp_fig/mutidimensional.png", bbox_inches="tight", dpi=1000)
plt.show()